# Mushroom Toxicity Classification Technical Report 


## Introduction

## Data Analysis

In [7]:
import importlib

import mysklearn.mypytable as mypytable
importlib.reload(mypytable)
from mysklearn.mypytable import MyPyTable

import mysklearn.myknnclassifier
importlib.reload(mysklearn.myknnclassifier)
from mysklearn.myknnclassifier import MyKNeighborsClassifier

import mysklearn.mynaivebayesclassifier
importlib.reload(mysklearn.mynaivebayesclassifier)
from mysklearn.mynaivebayesclassifier import MyNaiveBayesClassifier

import mysklearn.myeval
importlib.reload(mysklearn.myeval)
import mysklearn.myeval as myeval



import utils
importlib.reload(utils)
import utils as utils

mush_data = MyPyTable()
mush_data.load_from_file('/home/CPSC322finalProject/input_data/new_mushroom_cleaned.csv')

### KNN

In [8]:
# train test split and data set shrink
indexes_to_remove = utils.rand_inds(50000, 54034)
mush_data.drop_rows(indexes_to_remove)


#print(len(mush_data.data))

df_mush = []
for i, row in enumerate(mush_data.data):
    df_mush.append(row)

#print(len(df_mush))


season = [row[7] for row in df_mush]
stem_width = [row[5] for row in df_mush]
gill_color = [row[3] for row in df_mush]
cap_diameter = [row[0] for row in df_mush]

yummy_or_nah = [row[8] for row in df_mush]

# normalize the data
season_norm = utils.normalize_data(season)
stem_width_norm = utils.normalize_data(stem_width)
gill_color_norm = utils.normalize_data(gill_color)
cap_diameter_norm = utils.normalize_data(cap_diameter)

data_zipped = list(zip(season_norm, stem_width_norm, gill_color_norm, cap_diameter_norm))
X_data = data_zipped

X_train, X_test, y_train, y_test = myeval.train_test_split(X_data, yummy_or_nah, test_size=0.33, random_state=1, shuffle=True)
print(len(y_test))
print(len(X_test))
print(len(X_train))
print(len(y_train))

1332
1332
2703
2703


In [10]:
# initiate the knn
knn = MyKNeighborsClassifier(n_neighbors= 5)

knn.fit(X_train, y_train)

knn_y_pred = knn.predict(X_test)

In [11]:
acc = utils.compute_accuracy(knn_y_pred, y_test)
print(f'Accuacy: {acc:.4f}')

recall = utils.compute_recall(knn_y_pred, y_test, positive_label=1)
print(f'Recall: {recall:.4f}')

Accuacy: 0.7988
Recall: 0.8226


### Naive Bayes

In [12]:
y = [row[-1] for row in mush_data.data]   # Labels: survived
k = 10
# Stratified k-fold split
folds = myeval.stratified_kfold_split(X_data, y, n_splits=k, shuffle=True, random_state=2)


In [14]:
# Initialize Naive Bayes classifier
nb_classifier = MyNaiveBayesClassifier()

# List to store the results for each fold
nb_results = []

# Perform cross-validation
for train_indices, test_indices in folds:
    # Split the data into training and testing sets based on the current fold
    X_train = [X_data[i] for i in train_indices]
    y_train = [y[i] for i in train_indices]
    X_test = [X_data[i] for i in test_indices]
    y_test = [y[i] for i in test_indices]

    # Train the Naive Bayes classifier
    nb_classifier.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = nb_classifier.predict(X_test)

    # Store the results (true labels and predicted labels)
    nb_results.append((y_test, y_pred))

In [15]:
# Accuracy and Error Rate
nb_accuracy, nb_error_rate = utils.calculate_accuracy_error_rate(nb_results)
# Display the results
print(f"Accuracy:     {nb_accuracy:.4f}")
print(f"Error Rate:   {nb_error_rate:.4f}")

Accuracy:     0.6072
Error Rate:   0.3928
